In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet152V2
import pathlib

In [2]:
import os
base_dir = 'C:\Sem 5\Studi Independen\Machine Learning\CapstoneProject\datasetBersih'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

In [3]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,           # normalisasi data
                    horizontal_flip=True,     # data dapat berputar balik secara horizontal                  
                    ) 
test_datagen = ImageDataGenerator(
                    rescale=1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=64,
                                                    class_mode='categorical',
                                                    target_size=(224, 224),
                                                    color_mode= 'rgb', 
                                                    shuffle= True
                                                    )
test_generator = test_datagen.flow_from_directory(validation_dir,
                                                  batch_size=64,
                                                  class_mode='categorical',
                                                  target_size=(224, 224),
                                                  color_mode= 'rgb', 
                                                  shuffle= True
                                                  )

Found 11110 images belonging to 11 classes.
Found 2783 images belonging to 11 classes.


In [5]:
class_dict=train_generator.class_indices
classes=list(class_dict.keys())
print ('LIST OF CLASSES ', classes)
print ('CLASS DICTIONARY ',class_dict)
number_of_classes=len(classes) # this is the number of neurons in your top layer of the model
print ('Number of classes = ', number_of_classes)

LIST OF CLASSES  ['Bacterial_spot', 'Early_blight', 'Late_blight', 'Leaf_Mold', 'Septoria_leaf_spot', 'Spider_mites Two-spotted_spider_mite', 'Target_Spot', 'Tomato_Yellow_Leaf_Curl_Virus', 'Tomato_mosaic_virus', 'healthy', 'powdery_mildew']
CLASS DICTIONARY  {'Bacterial_spot': 0, 'Early_blight': 1, 'Late_blight': 2, 'Leaf_Mold': 3, 'Septoria_leaf_spot': 4, 'Spider_mites Two-spotted_spider_mite': 5, 'Target_Spot': 6, 'Tomato_Yellow_Leaf_Curl_Virus': 7, 'Tomato_mosaic_virus': 8, 'healthy': 9, 'powdery_mildew': 10}
Number of classes =  11


In [6]:
base_model = tf.keras.applications.efficientnet.EfficientNetB3(include_top= False, 
                                                               weights= "imagenet", 
                                                               input_shape= (224, 224, 3), 
                                                               pooling= 'max')

In [7]:
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),                                                # layer input untuk deep learning neural network 
    tf.keras.layers.Dense(11, activation= 'softmax')                                 # output layer 3 class
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb3 (Functional)  (None, 1536)             10783535  
                                                                 
 dense (Dense)               (None, 256)               393472    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 11)                2827      
                                                                 
Total params: 11,179,834
Trainable params: 11,092,531
Non-trainable params: 87,303
_________________________________________________________________


In [9]:
model.compile(optimizer ='Adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [10]:
class TestCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('accuracy') > 0.90 and logs.get('val_accuracy') > 0.90):
            print("\nAkurasi telah mencapai >90%!") 
            self.model.stop_training = True 
callbacks = TestCallback()

In [12]:
history=model.fit(
      train_generator,
      steps_per_epoch=4,                    
      epochs=25,                            
      validation_data=test_generator,       
      verbose=2,
      validation_steps=4,             
      callbacks = [callbacks]
      )

Epoch 1/25
4/4 - 128s - loss: 7.9191 - accuracy: 0.1914 - val_loss: 2.6735 - val_accuracy: 0.0977 - 128s/epoch - 32s/step
Epoch 2/25
4/4 - 118s - loss: 3.5821 - accuracy: 0.3086 - val_loss: 2.5289 - val_accuracy: 0.0977 - 118s/epoch - 29s/step
Epoch 3/25
4/4 - 115s - loss: 1.8390 - accuracy: 0.4531 - val_loss: 2.5153 - val_accuracy: 0.0977 - 115s/epoch - 29s/step
Epoch 4/25
4/4 - 115s - loss: 1.3407 - accuracy: 0.5547 - val_loss: 2.4457 - val_accuracy: 0.1211 - 115s/epoch - 29s/step
Epoch 5/25
4/4 - 114s - loss: 1.1075 - accuracy: 0.6250 - val_loss: 2.5743 - val_accuracy: 0.0938 - 114s/epoch - 28s/step
Epoch 6/25
4/4 - 115s - loss: 1.1685 - accuracy: 0.6367 - val_loss: 2.5209 - val_accuracy: 0.0938 - 115s/epoch - 29s/step
Epoch 7/25
4/4 - 113s - loss: 0.8895 - accuracy: 0.7305 - val_loss: 2.4675 - val_accuracy: 0.1055 - 113s/epoch - 28s/step
Epoch 8/25
4/4 - 113s - loss: 0.7621 - accuracy: 0.8008 - val_loss: 2.4220 - val_accuracy: 0.0938 - 113s/epoch - 28s/step
Epoch 9/25


UnknownError: Graph execution error:

OSError: image file is truncated
Traceback (most recent call last):

  File "C:\ProgramData\Anaconda3\lib\site-packages\PIL\ImageFile.py", line 239, in load
    s = read(self.decodermaxblock)

  File "C:\ProgramData\Anaconda3\lib\site-packages\PIL\PngImagePlugin.py", line 932, in load_read
    cid, pos, length = self.png.read()

  File "C:\ProgramData\Anaconda3\lib\site-packages\PIL\PngImagePlugin.py", line 177, in read
    length = i32(s)

  File "C:\ProgramData\Anaconda3\lib\site-packages\PIL\_binary.py", line 85, in i32be
    return unpack_from(">I", c, o)[0]

struct.error: unpack_from requires a buffer of at least 4 bytes for unpacking 4 bytes at offset 0 (actual buffer size is 0)


The above exception was the direct cause of the following exception:


Traceback (most recent call last):

  File "C:\Users\hanao\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy Model')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()